In [1]:
from servitium_cognitionis.llms.mock import LLMMockFamily
from servitium_cognitionis.llms.gemini import LLMGeminiFamily
from servitium_cognitionis.personas.persona_base import Persona

In [2]:
persona_dani = Persona.from_json(
    "../dados/personas/professores/redacao/dani-stella/persona_config.json"
)

In [3]:
llm_family = LLMGeminiFamily()
llm_model = llm_family.get_model(persona_dani.thought_process)

In [4]:
print(llm_model)

gemini-1.5-pro-preview-0409


In [5]:
llm_model.initialize_model(
    system_instruction=persona_dani.present_initial_state(),
    temperature=persona_dani.creativity_level,
    max_output_tokens=persona_dani.speech_conciseness,
)

In [6]:
session_id = 1
llm_model.create_chat(session_id)

In [7]:
from IPython.display import Markdown


def print_as_markdown(text):
    previous_line = ""
    for t, _ in text:
        lines = t.split("\n")
        # Concatenate previous line with the first line of the current text
        if previous_line:
            lines[0] = previous_line + lines[0]
        # Display all lines except the last one
        for line in lines[:-1]:
            if len(line) > 0:
                display(Markdown(line))
        # Store the last line for the next iteration
        previous_line = lines[-1]
    # Display the last line after the loop finishes
    if previous_line and len(previous_line) > 0:
        display(Markdown(previous_line))

In [8]:
print_as_markdown(
    llm_model.send_stream_single_message("olá querida professora, tudo bem?")
)

Olá! Tudo bem sim, e com você? 😊 

In [9]:
# create a method to open a file and load it as a string
def load_file_as_string(file_path):
    with open(file_path, "r") as file:
        return f"file: {file_path}:\n\n" + file.read()

In [10]:
!ls

dados_redacao.ipynb	 random_playground.ipynb  texto_a_proposta_1_2018.png
notion_playground.ipynb  text_extraction.ipynb


In [27]:
from vertexai.generative_models import Part, Image

redacao_unicamp_arquivos = [
    "Exemplo de formatação dos arquivos json: \n\n",
    load_file_as_string(
        "../dados/vestibulares/unicamp/redacao/2021/2021_CQ_RED-3_proposta_2.json"
    ),
    "\n\n----\n\n",
    load_file_as_string(
        "../dados/vestibulares/unicamp/redacao/2019/2019_proposta_1.md",
    ),
    "\n\n",
    "História em quadrinhos da coletânea 2: \n\n",
    Image.load_from_file(
        "../dados/vestibulares/unicamp/redacao/2019/imagem_coletanea_2_2019_proposta_1.png"
    ),
    "\n\n",
    load_file_as_string(
        "../dados/vestibulares/unicamp/redacao/2019/2019_proposta_1.json"
    ),
    """\n\n----\n
Dani, analise o arquivo da segunda proposta de 2021 acima para entender o formato dos arquivos json esperados
para as minhas redações. Então, analise o arquivo markdown da primeira proposta de 2019 acima para então 
finalizar a criação da seção de coletanea do arquivo json da primeira proposta de 2019 acima. Então 
dê como resposta o arquivo json finalizado, contudo apenas a parte da coletanea. Assegure que o conteudo preenchido para a seção de 'contexto_textual' seja uma copia exata do conteudo da respectiva coletanea no arquivo markdown. Adiciono acima também a história em quadrinhos das coletânea 2.
    """,
]

In [ ]:
results = llm_model.send_stream_chat_message(session_id, redacao_unicamp_arquivos)
print_as_markdown(results)

In [ ]:
str_stats = []

In [32]:
# code to extract all the text content from a pdf
results = llm_model.send_stream_single_message(
    "generate a python code to extract all the text content from a pdf. Forneça o código completo em python"
)

In [31]:
!pip install pypdf2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 670.1 kB/s eta 0:00:001m695.7 kB/s eta 0:00:01


In [35]:
import PyPDF2


def extrair_pdf(file_path):
    with open(file_path, "rb") as arquivo_pdf:
        leitor_pdf = PyPDF2.PdfReader(arquivo_pdf)
        texto = ""
        for num_pagina in range(len(leitor_pdf.pages)):
            pagina = leitor_pdf.pages[num_pagina]
            texto += pagina.extract_text()
        return texto

In [39]:
pdf_content = extrair_pdf(
    "../dados/vestibulares/unicamp/redacao/2018/Relatorio_2_Fase_site.pdf"
)